In [0]:
from pyspark.sql.types import DateType,StructField,StructType,TimestampType
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
spark = SparkSession.builder.getOrCreate()
schema = StructType(
    [
        StructField("min_date", DateType()),
        StructField("end_date", TimestampType())
    ]
)

df = spark.createDataFrame(
    [("2025-12-06", "2025-12-08 07:17:57.37546")], 
    ["min_date", "end_date"]
).withColumn("min_date", col("min_date").cast("date")) \
 .withColumn("end_date", col("end_date").cast("timestamp"))


In [0]:
from pyspark.sql import functions as F
display(df)
date_seq = df.select(
    F.sequence(
        F.to_date(F.col("min_date")),
        F.to_date(F.col("end_date")),
        F.expr("interval 1 DAY")
    ).alias("cal_date")
)

date_seq = date_seq.select(F.explode("cal_date"))


display(date_seq)

In [0]:
from pyspark.sql import functions as F
F.col("min_date").between(
    F.date_add(F.col("end_date"),-6), F.col("end_date")
)

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

spark = SparkSession.builder.getOrCreate()

schema = StructType([
    StructField("emp_id", IntegerType(), True),
    StructField(
        "emp_details",
        StructType([
            StructField("name", StringType(), True),
            StructField("dept", StringType(), True),
            StructField("city", StringType(), True)
        ]),
        True
    )
])

data = [
    (1, ("Mukil", "Data", "Chennai")),
    (2, ("Rahul", "Analytics", "Bangalore")),
    (3, ("Anu", "Engineering", "Hyderabad"))
]

df = spark.createDataFrame(data, schema)
df.show(truncate=False)
df.printSchema()


In [0]:
df.write \
  .format("delta") \
  .mode("overwrite") \
  .saveAsTable("databricks_project_practice_sxm.default.emp_struct_delta")


In [0]:
display(df)

In [0]:
%sql
CREATE TABLE databricks_project_practice_sxm.default.emp_struct_delta_tgt
USING DELTA
AS
SELECT
    emp_id,
    named_struct(
        'name', upper(emp_details.name),
        'dept', emp_details.dept,
        'city', emp_details.city
    ) AS emp_details
FROM databricks_project_practice_sxm.default.emp_struct_delta;


In [0]:
%sql
select * from databricks_project_practice_sxm.default.emp_struct_delta_tgt

In [0]:
%sql
DROP TABLE IF EXISTS default.tgt_struct;

CREATE TABLE default.tgt_struct (
  id INT,
  details STRUCT<
    name: STRING,
    dept: STRING
  >
)
USING DELTA;


In [0]:
%sql
DROP TABLE IF EXISTS default.src_struct;

CREATE TABLE default.src_struct (
  id INT,
  details STRUCT<
    name: STRING,
    dept: STRING,
    city: STRING   -- 🔥 extra field
  >
)
USING DELTA;


In [0]:
%sql
INSERT INTO default.src_struct VALUES
(1, named_struct('name','Priya','dept','Data','city','Chennai'));


In [0]:
%sql
INSERT INTO default.tgt_struct
SELECT
  id,
  details
FROM default.src_struct;
